# Chpt 1: Financial Data Structures

## Standard Bars

### Time Bars -sampled on min, hour, day etc:
* Timestamp 
* Volume-weighted average price = avg price a security has traded at throughout the day based on both volume and price = (price * volume) / total volume. numerator is for every transaction
* Open price
* Close price
* High price
* Low price
* Volume Trades

Pros:
* Extremely easy to obtain

Cons: 
* Markets dont process info at constant time intervals. Mornings are more active etc
* Time bars oversample low-activity periods and undersample high-activity periods
* Shit statistical properties
    * Serial Correlation: Errors in a given time period carry forward. Ex: overestimate in one year high dividends will carry over to the next
    * Heteroscedasticity: The size of the error term differs across values of an independent variable
    * Non-normality of returns 
    
    

### Tick Bars - sampled every x num of trades
* Take a sample variable (timestamp, VWAP, open price etc) each time x **number of transactions (trades)** take place
    * EX: Sample VWAP every 1000 trades (ticks) 
Pros:
* Sampling as a function of number of transactions lets the returns be more IID (independent and identically distributed normal), which allows us to make gaussian assumptions
* Instead of depending on each minute or each day, is instead based on every 1000 trades which better fits info processing of markets

Cons:
* Arbitariness in number of ticks
* Ex: One order sitting in offer for 10 lots is one tick, but 10 orders of 1 lot are 10 ticks

### Volume Bars - sampled every X amount of volume
* Same as tick, but the variable is sampled every time X volume has been traded
Pros:
* Naturally describes relationship between volume and price (foundation for market microstructure theories)

### Dollar Bars - sample every X amount of dollars exchanged
* Represents the fact that number of shares traded is a function of actual value exchanged (if a stock price goes up 100% selling $1000 of the stock will require only selling half the num of shares as you bought)
* When analysis involves signifigant price fluctuation, use dolar bars
* Is robust to splits, buybacks etc which would impact volume bars

In [2]:
# Markdown guide Latex: https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd

## Information Driven-Bars

### Tick Imbalance Bars - TIBS
* Sequence of ticks $ \left\{(p_{t} , v_{t}) \right\}_{t=1,...,T}$
* $ p_{t} $ is price associated with tick $ t $. Tick is minimum upward or downward movement in price of a security (transaction / trade)

#### To Sign Ticks with Tick Rule:
* **Tick Rule** - how to sign ticks to find imbalance:
    * A sequence $ \left\{b_{t}\right\}_{t=1,...,T}$ where:

$$
b_{t} = \left\{
    \begin{array}\\
         b_{t-1} & \mbox{if } \Delta p_{t} = 0 \\
        \frac{|\Delta p_{t}|}{\Delta p_{t}} \ & \mbox{if } \ \Delta p_{t} \neq 0 \\
    \end{array}
\right.
$$

* This rule allows us to assign a value $ b_{t} \in \left\{ -1,1 \right\} $ 
* This is "signing" the ticks based on change in price
* The goal is to find a **tick imbalance** hence creating the Tick Imbalance Bars **TIBs** to sample whenever a tick imbalance exceeds expectations. 
* A tick imblance is given by the accumulation (sum) of signed ticks, to tick index $ T $. The accumulation will either be negative or positve, exceeding expectation and giving info

##### To find Tick Imblance at time $ T $:
* Sum of tick signs in period of time T: $\theta_{T} $ = tick imbalance at time T ( Basically how positive (bullish) or negative (bearish) the sum of the ticks are in the period)
$$
\theta_{T} = \sum_{t=1}^{T} b_{t}$$
How to determine $ T $

1. Define Tick Imbalance at time T as $ \theta_{T} = \sum_{t=1}^{T} b_{t} $
2. Compute the expected value of the tick imbalance (how positive or neg its expected to be) with
    * $ E_{0}[\theta_{T}]= E_{0}[T](2P[b_{t}=1] - 1)$
    * ^ Can be broken down as Exp Value of Tick Imbalance = Expected size of tick bar * probabilities the tick was a buy or a sell
    * In practice: $ E_{0}[\theta_{T}] $ = avg size of previous bars * avg $b_{t} $ (value of sign of previous ticks) 
    
3. Define size: A tick imbalance bar (TIB) as a T*-contiguous subset of ticks such that the following condition is met:
    * $ T* = \underset{T}{\operatorname{argmin}} \left\{ |\theta_{T}|  \geq  E_{0}[T]|2P[b_{t}=1] - 1| \right\}$ 
    * Basically, Take the minimum size T where the tick imbalance $ \theta_{T}$ for group of Ticks size T is $ \geq $ expected value of the expected value of the tick imbalance for size T group of past

#### Summary:
* **Process**
    1. Take in Raw ticks
    2. Sign all the raw ticks with the tick rule
    3. Find correct size T to make bar with
* **Result**
    * A **TIB (tick imbalance bar)** where each bar is a T size group of ticks where the tick imbalance (how + or - sum of signs is ) for size T is greater than the expected value for size a TIB size T (using past tick averages). Basically, **sample a group of ticks using the smallest size T (creating a TIB) when the tick imbalance is greater than expected for that size**
    * Lower T = more samples
    * A TIB is a bucket of trades containing equal information (regardless of volume, price, etc)
    * When $ \theta_{T} $ is more imbalanced, a smaller bucket size will be required (smaller T) and TIBS are generated more frequently. Hence TIBs are produced more frequently in the presence of informed trading (one-side imbalanced trading based on asymetric info traders have), and data sampling is synched with arrival of informed traders
 